In [ ]:
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
torch.use_deterministic_algorithms(True) # Needed for reproducible results

Random Seed:  999


In [ ]:
import pandas as pd

In [ ]:
torch_version = str(torch.__version__)
print(f'torch version: {torch_version}')

scatter_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
sparse_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
%pip install torch-scatter -f $scatter_src
%pip install torch-sparse -f $sparse_src
%pip install torch-geometric
%pip install ogb

torch version: 2.1.0+cu121
Looking in links: https://pytorch-geometric.com/whl/torch-2.1.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 102.1 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.1.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7026 sha256=2e0da9b8f7cec15e2fac0faecc55f08cdd7222001b13db5615367c8805c00590
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built littleutils


In [ ]:
# Mount your google drive in google colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
elif USE_GPU and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
print(device)

cpu


In [ ]:
MOVIE_HEADERS = [
    "movieId", "title", "releaseDate", "videoReleaseDate", "IMDb URL",
    "unknown", "Action", "Adventure", "Animation", "Children's", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
    "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"
]
USER_HEADERS = ["userId", "age", "gender", "occupation", "zipCode"]
RATING_HEADERS = ["userId", "movieId", "rating", "timestamp"]


data_path = '/content/drive/MyDrive/CS247/Project/ColdGAN/ml-100k/'

# Process user data:
df_user = pd.read_csv(
    #Path to user data goes here
    data_path + 'u.user',
    sep='|',
    header=None,
    names=USER_HEADERS,
    index_col='userId',
    encoding='ISO-8859-1',
)


# Process rating data for training:
df_rating_train = pd.read_csv(
    data_path + 'u1.base',
    sep='\t',
    header=None,
    index_col='userId',
    names=RATING_HEADERS,
).reset_index()

# Process rating data for testing:
df_rating_test = pd.read_csv(
    data_path + 'u1.test',
    sep='\t',
    header=None,
    index_col='userId',
    names=RATING_HEADERS,
).reset_index()

#Combine user and rating data into one vector

# df_train = df_user.copy().merge(df_rating_train, how='right', left_on='userId', right_on='userId')

# df_test = df_user.copy().merge(df_rating_test, how='right', left_on='userId', right_on='userId')
# rating = torch.from_numpy(df_rating_train['rating'].values).to(torch.long)


In [ ]:
user_mapping = {idx: i for i, idx in enumerate(df_user.index)}

age = df_user['age'].values / df_user['age'].values.max()
age = torch.from_numpy(age).to(torch.float).view(-1, 1)

gender = df_user['gender'].str.get_dummies().values
gender = torch.from_numpy(gender).to(torch.float)

occupation = df_user['occupation'].str.get_dummies().values
occupation = torch.from_numpy(occupation).to(torch.float)

zipcode = df_user['zipCode'].str.get_dummies().values
zipcode = torch.from_numpy(zipcode).to(torch.float)

users = torch.cat([age, gender, occupation, zipcode], dim=-1).to(device)

In [ ]:
ratings = []
timestamps = []
num_movies = 1682
for user in user_mapping:
  user_ratings_df = df_rating_train.loc[df_rating_train['userId'] == user].sort_values('timestamp')
  user_ratings = torch.zeros((num_movies,), dtype=torch.float)
  user_timestamps = torch.zeros((num_movies,), dtype=torch.float)
  for i, (index, row) in enumerate(user_ratings_df.iterrows()):
    user_ratings[row['movieId']-1] = row['rating']
    user_timestamps[row['movieId']-1] = i + 1



  ratings.append(user_ratings)
  timestamps.append(user_timestamps)

ratings = torch.from_numpy(np.array(ratings)).to(device)
timestamps = torch.from_numpy(np.array(timestamps))

In [ ]:
movie_popularity = df_rating_train['movieId'].value_counts().sort_index()
movie_popularity = movie_popularity.reindex(list(range(1,num_movies+1)),fill_value=0).values
movie_popularity = torch.from_numpy(movie_popularity / np.max(movie_popularity)).to(device)

In [ ]:
print(movie_popularity)

tensor([0.7913, 0.2169, 0.1550,  ..., 0.0021, 0.0021, 0.0021],
       dtype=torch.float64)


In [166]:
print(len(ratings))

943


In [167]:
print(len(users))

943


In [ ]:
len(movie_popularity)

1682

In [ ]:
from torch.utils.data import Dataset, DataLoader


class RatingsDataset(Dataset):
    def __init__(self, users, ratings, timestamps):
        self.len = len(users)
        self.users = users
        self.ratings = ratings
        self.timestamps = timestamps

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        user_vec = self.users[idx]
        ratings_vec = self.ratings[idx]
        timestamps_vec = self.timestamps[idx]

        return user_vec, ratings_vec, timestamps_vec

    @staticmethod
    def collate_fn(data):
        user_vec = torch.stack([_[0] for _ in data], dim=0)
        ratings_vec = torch.stack([_[1] for _ in data], dim=0)
        timestamps_vec = torch.stack([_[2] for _ in data], dim=0)
        return user_vec, ratings_vec, timestamps_vec

In [175]:
batch_size = 41
dataset = RatingsDataset(users, ratings, timestamps)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=RatingsDataset.collate_fn)

In [178]:
# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 41

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 100

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparameter for Adam optimizers
beta1 = 0.5


In [ ]:

# Get batch data from training set
def get_batch_data(file, index, size):  # 1,5->1,2,3,4,5
    user = []
    item = []
    label = []
    for i in range(index, index + size):
        line = linecache.getline(file, i)
        line = line.strip()
        line = line.split()
        user.append(int(line[0]))
        user.append(int(line[0]))
        item.append(int(line[1]))
        item.append(int(line[2]))
        label.append(1.)
        label.append(0.)
    return user, item, label

def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1


# Get category of items
def get_category(file_in):
    category = {}
    #with open(file_in) as fin:
    with open(file_in,encoding='unicode_escape') as fin:
        for line in fin:
            line = line.split('|')
            iid = int(line[0]) - 1  # item id starts from 0
            category[iid] = line[6:24]
    return category


# Get training/testing data
def get_train_test_data(file_in):
    # only record user-item pairs with rating >=4
    user_item = {}
    with open(file_in) as fin:
        for line in fin:
            line = line.split()
            uid = int(line[0])
            iid = int(line[1])
            r = float(line[2])
            if uid in user_item:
                user_item[uid].append(iid)
            else:
                user_item[uid] = [iid]
    return user_item


def precision_at_k(r, k):
    """Score is precision @ k
    Relevance is binary (nonzero is relevant).
    Returns:
        Precision @ k
    Raises:
        ValueError: len(r) must be >= k
    """
    assert k >= 1
    r = np.asarray(r)[:k]
    return np.mean(r)


def average_precision(r):
    """Score is average precision (area under PR curve)
    Relevance is binary (nonzero is relevant).
    Returns:
        Average precision
    """
    r = np.asarray(r)
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)


def mean_average_precision(rs):
    """Score is mean average precision
    Relevance is binary (nonzero is relevant).
    Returns:
        Mean average precision
    """
    return np.mean([average_precision(r) for r in rs])


def dcg_at_k(r, k):
    """Score is discounted cumulative gain (dcg)
    Relevance is positive real values.  Can use binary
    as the previous methods.
    Returns:
        Discounted cumulative gain
    """
    r = np.asfarray(r)[:k]
    if r.size:
        # if method == 0:
        #     return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        # elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        # else:
        #     raise ValueError('method must be 0 or 1.')
    else:
        return 0.


def ndcg_at_k(r, k):
    """Score is normalized discounted cumulative gain (ndcg)
    Relevance is positive real values.  Can use binary
    as the previous methods.
    Returns:
        Normalized discounted cumulative gain
    """
    dcg_max = dcg_at_k(sorted(r, reverse=True), k)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k) / dcg_max


def recall_at_k(r, k, all_pos_num):
    r = np.asfarray(r)[:k]
    return np.sum(r) / all_pos_num


def F1(pre, rec):
    if pre + rec > 0:
        return (2.0 * pre * rec) / (pre + rec)
    else:
        return 0.


def diversity_by_category(selected_items, item_cate, cate_num):
    cate = []
    for iid in selected_items:
        try:
            cate.append(item_cate[iid])
        except KeyError:
            pass

    cate_count = np.count_nonzero(np.sum(np.asarray(cate, np.float32), axis=0))

    return cate_count/cate_num


def get_div_train_data(file_in):
    user_train_samples = {}

    with open(file_in) as fin:
        for line in fin:
            line = line.split('\t')
            uid = int(line[0])
            items = list(map(int, line[1:]))
            if uid in user_train_samples:
                user_train_samples[uid].append(items)
            else:
                user_train_samples[uid] = [items]

    return user_train_samples


def generate_pairwise_diversity_training_data(file_train, file_cate, file_out_pos, file_out_neg, user_num):
    pos_data = []  # data for output
    neg_data = []

    #########################################################################################
    # Load data
    #########################################################################################
    category = get_category(file_cate)
    user_item = get_train_test_data(file_train)

    # for each user, generate diversity set
    for i in range(0, user_num):
        uid = i;
        print('user:', uid)
        try:
            items = user_item[uid]
        except KeyError:
            pass
        # the number of trials for each user is set to be the number of viewed items
        for j in range(0, len(items)):
            first_item = items[j]
            pos_div_set = [first_item]  # make sure each viewed item is sampled
            pos_cate = [category[first_item]]
            num_cate = np.count_nonzero(np.sum(np.asarray(pos_cate, np.float32), axis=0))
            # the number of trials for each diversity set is the number of viewed items
            for k in range(0, len(items)):
                new_item = np.random.choice(items)
                try:
                    pos_cate.append(category[new_item])
                    new_num_cate = np.count_nonzero(np.sum(np.asarray(pos_cate, np.float32), axis=0))
                    if new_num_cate - num_cate > 0:
                        pos_div_set.append(new_item)
                        num_cate = new_num_cate
                    if len(pos_div_set) == 10:
                        break;
                except KeyError:
                    pass

            pos_div_set.sort()
            pos_data.append(str(uid) + '\t' + '\t'.join(str(x) for x in pos_div_set))

            neg_div_set = [first_item]  # make sure each viewed item is sampled
            neg_cate = np.asarray(category[first_item], np.int32).nonzero()[0]
            # the number of trials for each diversity set is the number of viewed items
            for k in range(0, len(items)):
                new_item = items[k]
                try:
                    if new_item not in neg_div_set:
                        new_cate = np.asarray(category[new_item], np.int32).nonzero()[0]
                        if np.array_equal(neg_cate, new_cate):
                            neg_div_set = np.append(neg_div_set, new_item)
                            if len(neg_div_set) > 10:  # due to tensorflow bug
                                neg_div_set = np.random.choice(neg_div_set, 10, replace=False)
                except KeyError:
                    pass
            neg_div_set.sort()
            neg_data.append(str(uid) + '\t' + '\t'.join(str(x) for x in neg_div_set))

    with open(file_out_pos, 'w')as fout:
        fout.write('\n'.join(pos_data))

    with open(file_out_neg, 'w')as fout:
        fout.write('\n'.join(neg_data))


In [ ]:
# custom weights initialization called on ``netG`` and ``netD``
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
# Generator Code

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.ReLU(inplace=True))
            return layers

        self.main = nn.Sequential(
            *block(num_movies, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, num_movies)
        )

    def forward(self, user_vec, rating_vec):
        return self.main(rating_vec)

In [ ]:
ngpu = 1

In [ ]:
# Create the generator
netG = Generator(ngpu).to(device)

# Handle multi-GPU if desired
# if (device == 'cuda') and (ngpu > 1):
#     netG = nn.DataParallel(netG, list(range(ngpu)))

# # Apply the ``weights_init`` function to randomly initialize all weights
# #  to ``mean=0``, ``stdev=0.02``.
# netG.apply(weights_init)

# Print the model
print(netG)

Generator(
  (main): Sequential(
    (0): Linear(in_features=1682, out_features=128, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): BatchNorm1d(256, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (4): ReLU(inplace=True)
    (5): Linear(in_features=256, out_features=512, bias=True)
    (6): BatchNorm1d(512, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU(inplace=True)
    (8): Linear(in_features=512, out_features=1024, bias=True)
    (9): BatchNorm1d(1024, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Linear(in_features=1024, out_features=1682, bias=True)
  )
)


In [ ]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.Linear(num_movies, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )


    def forward(self, user_vec, rating_vec):
        return self.main(rating_vec)

In [ ]:
# Create the Discriminator
netD = Discriminator(ngpu).to(device)

# Handle multi-GPU if desired
# if (device == 'cuda') and (ngpu > 1):
#     netD = nn.DataParallel(netD, list(range(ngpu)))

# # Apply the ``weights_init`` function to randomly initialize all weights
# # like this: ``to mean=0, stdev=0.2``.
# netD.apply(weights_init)

# Print the model
print(netD)

Discriminator(
  (main): Sequential(
    (0): Linear(in_features=1682, out_features=512, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=256, out_features=1, bias=True)
    (5): Sigmoid()
  )
)


In [ ]:
# Initialize the ``BCELoss`` function
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
# fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [187]:
#Generate cold state from warm state
p_min = 0.0
p_max = 0.5
alpha = 1
def rejuvenation_function(rating_vector, timestamp_vector, alpha):
  #Need time + popularity of item rating
  #For tth item, probability of choosing in cold state is
  #pm(t) = p_min + (p_max - p_min) * exp(-alpha * [t - pop(i_t)]/ [count(wm)])
  count = np.count_nonzero(rating_vector)
  t_vector = timestamp_vector
  prob_vector = p_min + (p_max - p_min) * np.exp(-alpha * (t_vector - movie_popularity)/ count)
  random_selection = torch.from_numpy(np.random.rand(num_movies)).to(device) < prob_vector
  return rating_vector * random_selection

In [188]:
warm_vector = ratings[0]
cold_vector = rejuvenation_function(ratings[0], timestamps[0], alpha)
for i in range(num_movies):
  if warm_vector[i] != cold_vector[i]:
    print(warm_vector[i])
    print(cold_vector[i])

tensor(3.)
tensor(0.)
tensor(4.)
tensor(0.)
tensor(3.)
tensor(0.)
tensor(3.)
tensor(0.)
tensor(1.)
tensor(0.)
tensor(5.)
tensor(0.)
tensor(2.)
tensor(0.)
tensor(5.)
tensor(0.)
tensor(4.)
tensor(0.)
tensor(5.)
tensor(0.)
tensor(1.)
tensor(0.)
tensor(4.)
tensor(0.)
tensor(4.)
tensor(0.)
tensor(1.)
tensor(0.)
tensor(3.)
tensor(0.)
tensor(2.)
tensor(0.)
tensor(2.)
tensor(0.)
tensor(3.)
tensor(0.)
tensor(3.)
tensor(0.)
tensor(2.)
tensor(0.)
tensor(5.)
tensor(0.)
tensor(4.)
tensor(0.)
tensor(4.)
tensor(0.)
tensor(5.)
tensor(0.)
tensor(5.)
tensor(0.)
tensor(5.)
tensor(0.)
tensor(5.)
tensor(0.)
tensor(5.)
tensor(0.)
tensor(2.)
tensor(0.)
tensor(4.)
tensor(0.)
tensor(4.)
tensor(0.)
tensor(4.)
tensor(0.)
tensor(4.)
tensor(0.)
tensor(4.)
tensor(0.)
tensor(5.)
tensor(0.)
tensor(2.)
tensor(0.)
tensor(4.)
tensor(0.)
tensor(3.)
tensor(0.)
tensor(4.)
tensor(0.)
tensor(1.)
tensor(0.)
tensor(5.)
tensor(0.)
tensor(3.)
tensor(0.)
tensor(3.)
tensor(0.)
tensor(4.)
tensor(0.)
tensor(1.)
tensor(0.)
tensor(4.)

In [189]:
def relevant_loss(rating_vector, actual_warm):
  #Sum of binary cross-entropy loss b/w sigmoid gan out and wrel
  #divided by number items rated by user
  n_m = torch.from_numpy(np.count_nonzero(actual_warm, axis=1)).view(-1, 1)
  avg_rating = torch.from_numpy(np.average(actual_warm, axis=1)).view(-1, 1)
  relevance_vector = (actual_warm > avg_rating).float()
  activation_vector = (actual_warm > 0).float()

  bce_loss = nn.functional.binary_cross_entropy(
      nn.functional.sigmoid(rating_vector), relevance_vector, reduction='none')
  loss = torch.sum(bce_loss * activation_vector) / n_m

  return loss

In [190]:
for i in dataloader:
  cold = rejuvenation_function(i[1], i[2], alpha)
  print(relevant_loss(cold, i[1]).sum())
  break

tensor(1541.2250)


In [200]:
# Training Loop

# Lists to keep track of progress
fake_warm = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")
print(dataloader)
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(dataloader):
        user_vecs = data[0]
        ratings_vecs = data[1]
        timestamps_vecs = data[2]
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        netD.zero_grad()
        # Format batch
        b_size = user_vecs.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        # Forward pass real batch through D
        output = netD(user_vecs, ratings_vecs).view(-1)
        # Calculate loss on all-real batch
        errD_real = criterion(output, label)
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        cold_vectors = rejuvenation_function(ratings_vecs, timestamps_vecs, alpha)
        # Generate fake image batch with G
        fake  = netG(user_vecs, cold_vectors)
        label.fill_(fake_label)
        # Classify all fake batch with D
        output = netD(user_vecs, fake).view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, label)
        # Calculate the gradients for this batch, accumulated (summed) with previous gradients
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Compute error of D as sum over the fake and the real batches
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        #Generate fakes again
        fake  = netG(user_vecs, cold_vectors)
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(user_vecs, fake).view(-1)
        # Calculate G's loss based on this output
        rel_loss = 0 #relevant_loss(fake, ratings_vecs).sum() / np.count_nonzero(ratings_vecs)
        errG = criterion(output, label) + rel_loss
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()

        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            #Get random cold vector
            idx = np.random.randint(len(users), size=2)
            noise_user = users[idx]
            fixed_noise = rejuvenation_function(ratings[idx], timestamps[idx], alpha=0.1)
            with torch.no_grad():
                fake = netG(noise_user, fixed_noise)
            fake_warm.append(fake)

        iters += 1

Starting Training Loop...
[0/100][0/23]	Loss_D: 1.4190	Loss_G: 0.7234	D(x): 0.4701	D(G(z)): 0.4851 / 0.4851
[1/100][0/23]	Loss_D: 1.4240	Loss_G: 0.7234	D(x): 0.4679	D(G(z)): 0.4851 / 0.4851
[2/100][0/23]	Loss_D: 1.4188	Loss_G: 0.7234	D(x): 0.4702	D(G(z)): 0.4851 / 0.4851
[3/100][0/23]	Loss_D: 1.4249	Loss_G: 0.7234	D(x): 0.4675	D(G(z)): 0.4851 / 0.4851
[4/100][0/23]	Loss_D: 1.4221	Loss_G: 0.7234	D(x): 0.4688	D(G(z)): 0.4851 / 0.4851
[5/100][0/23]	Loss_D: 1.4296	Loss_G: 0.7234	D(x): 0.4651	D(G(z)): 0.4851 / 0.4851
[6/100][0/23]	Loss_D: 1.4256	Loss_G: 0.7234	D(x): 0.4671	D(G(z)): 0.4851 / 0.4851
[7/100][0/23]	Loss_D: 1.4155	Loss_G: 0.7234	D(x): 0.4718	D(G(z)): 0.4851 / 0.4851
[8/100][0/23]	Loss_D: 1.4179	Loss_G: 0.7234	D(x): 0.4706	D(G(z)): 0.4851 / 0.4851
[9/100][0/23]	Loss_D: 1.4301	Loss_G: 0.7234	D(x): 0.4651	D(G(z)): 0.4851 / 0.4851
[10/100][0/23]	Loss_D: 1.4206	Loss_G: 0.7234	D(x): 0.4694	D(G(z)): 0.4851 / 0.4851
[11/100][0/23]	Loss_D: 1.4171	Loss_G: 0.7234	D(x): 0.4710	D(G(z)): 0.48